[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weave/examples/blob/master/examples/cookbooks/weave_litellm_integration_docs.ipynb)
<!--- @wandbcode{weave-litellm-integration} -->

In [ ]:
%%capture
!pip install wandb weave litellm

In [2]:
try:
    from google.colab import userdata
    import os
    os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
except:
    from dotenv import load_dotenv
    load_dotenv

In [4]:
project = "weave_litellm_integration"

In [5]:
import litellm

In [6]:
import weave
weave.init(project)

Logged in as Weights & Biases user: a-sh0ts.
View Weave data at https://wandb.ai/a-sh0ts/litellm_weave_integration/weave


In [7]:
response = litellm.completion(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Translate 'Hello, how are you?' to French"}],
    max_tokens=1024
)
print(response.choices[0].message.content)

🍩 https://wandb.ai/a-sh0ts/litellm_weave_integration/r/call/87b74fc6-511a-4a3c-a4a2-80192c5c954a
Bonjour, comment vas-tu ?


In [8]:
response = litellm.completion(
    model="claude-3-5-sonnet-20240620",
    messages=[{"role": "user", "content": "Translate 'Hello, how are you?' to French"}],
    max_tokens=1024
)
print(response.choices[0].message.content)

🍩 https://wandb.ai/a-sh0ts/litellm_weave_integration/r/call/3b79d7ca-e435-4ed1-b8a0-c35853d5d5f2
The French translation of "Hello, how are you?" is:

"Bonjour, comment allez-vous?"

This is the formal way of asking. If you want a more informal version, you could say:

"Salut, ça va?"


In [9]:
@weave.op()
def translate(text: str, target_language: str, model: str) -> str:
    response = litellm.completion(
        model=model,
        messages=[{"role": "user", "content": f"Translate '{text}' to {target_language}"}],
        max_tokens=1024
    )
    return response.choices[0].message.content

print(translate("Hello, how are you?", "French", "gpt-3.5-turbo"))
print(translate("Hello, how are you?", "Spanish", "claude-3-5-sonnet-20240620"))

🍩 https://wandb.ai/a-sh0ts/litellm_weave_integration/r/call/155100ed-cd80-4790-aa22-a8635a923b16
Bonjour, comment vas-tu ?
🍩 https://wandb.ai/a-sh0ts/litellm_weave_integration/r/call/6acab84e-7227-44ae-999f-58f30f276b84
The Spanish translation of "Hello, how are you?" is:

"Hola, ¿cómo estás?"

This is the informal version, which you would use with friends, family, or people you're familiar with. If you need a more formal version, you could say:

"Hola, ¿cómo está usted?"

Both versions mean the same thing, but the second one is more polite and used in formal situations or when addressing someone older or in a position of authority.


In [10]:
class TranslatorModel(weave.Model):
    model: str
    temperature: float

    @weave.op()
    def predict(self, text: str, target_language: str):
        response = litellm.completion(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are a translator. Translate the given text to {target_language}."},
                {"role": "user", "content": text}
            ],
            max_tokens=1024,
            temperature=self.temperature
        )
        print(response)
        print(response.choices[0].message.content)
        print(type(response.choices[0].message.content))
        return response.choices[0].message.content

# Create instances with different models
gpt_translator = TranslatorModel(model="gpt-3.5-turbo", temperature=0.3)
claude_translator = TranslatorModel(model="claude-3-5-sonnet-20240620", temperature=0.1)


# Use different models for translation
english_text = "Hello, how are you today?"

print("GPT-3.5 Translation to French:")
print(gpt_translator.predict(english_text, "French"))

print("\nClaude-3.5-Sonnet Translation to Spanish:")
print(claude_translator.predict(english_text, "Spanish"))

GPT-3.5 Translation to French:
ModelResponse(id='chatcmpl-9imYE3ZGUe3xBu8a0DCSQfHvJvwtL', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Bonjour, comment vas-tu aujourd'hui ?", role='assistant'))], created=1720460090, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=Usage(completion_tokens=9, prompt_tokens=30, total_tokens=39))
Bonjour, comment vas-tu aujourd'hui ?
<class 'str'>
🍩 https://wandb.ai/a-sh0ts/litellm_weave_integration/r/call/4b4a459b-1eaf-4366-b280-83ba47bc3e91
Bonjour, comment vas-tu aujourd'hui ?

Claude-3.5-Sonnet Translation to Spanish:
ModelResponse(id='chatcmpl-6aec83c8-3a4c-426b-8ab7-4275c69a11a5', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Hola, ¿cómo estás hoy?', role='assistant', tool_calls=[]))], created=1720460093, model='claude-3-5-sonnet-20240620', object='chat.completion', system_fingerprint=None, usage=Usage(prompt_tokens=27, completion_tokens=17, total_tokens=44))
H

In [11]:
claude_translator = TranslatorModel(model="claude-3-5-sonnet-20240620", temperature=0.1)
print(claude_translator.predict(english_text, "Spanish"))

ModelResponse(id='chatcmpl-33e030a8-4c59-4a2f-915e-4cbcb520cea4', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Hola, ¿cómo estás hoy?', role='assistant', tool_calls=[]))], created=1720460095, model='claude-3-5-sonnet-20240620', object='chat.completion', system_fingerprint=None, usage=Usage(prompt_tokens=27, completion_tokens=17, total_tokens=44))
Hola, ¿cómo estás hoy?
<class 'str'>
🍩 https://wandb.ai/a-sh0ts/litellm_weave_integration/r/call/ca153155-f956-4c9b-ab40-fa1edd27c384
Hola, ¿cómo estás hoy?


In [12]:
response = litellm.completion(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Translate 'Hello, how are you?' to French"}],
    functions=[
        {
            "name": "translate",
            "description": "Translate text to a specified language",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "The text to translate",
                    },
                    "target_language": {
                        "type": "string",
                        "description": "The language to translate to",
                    }
                },
                "required": ["text", "target_language"],
            },
        },
    ],
)

print(response.choices[0].message.function_call)

🍩 https://wandb.ai/a-sh0ts/litellm_weave_integration/r/call/b849c29e-6965-4d28-a0a5-a78d798483ea
FunctionCall(arguments='{"text":"Hello, how are you?","target_language":"fr"}', name='translate')
